# Enunciado
Parte 2 El metro de Londres (Underground) es el sistema de transporte ferroviario subterráneo más antiguo del mundo. Tiene 270 estaciones, 408 kilómetros de líneas, mueve más de 3 millones de personas diariamente y es un sistema crucial para el funcionamiento de una de las ciudades más importantes del mundo. Transport for London, la empresa que se encarga de operar el metro, desea realizar la valoración de la infraestructura para anticiparse a problemas o planes de mantenimiento que puedan presentarse durante la operación de la red. Se requiere que dicha valoración sea respaldada por datos y análisis. Utilizando análisis de Grafos y Redes se solicita contestar las siguientes preguntas:

Si se desea saber cuál es la estación más importante dentro de la red, ¿cuál o cuáles métricas utilizaría y por qué?
Utilizando la información del punto anterior indique ¿cuál o cuáles estaciones son las más importantes de la red?
Si se desea entender el impacto que tiene perder o deshabilitar una estación de la red ¿cuál o cuáles métricas utilizaría y por qué?
Utilizando las métricas seleccionadas demuestre como podría medirse el impacto de perder o deshabilitar una estación de la red de metro de Londres
Opcional, (realizar este punto le da un 30% más de valoración en su calificación final) de acuerdo con los conceptos analizados anteriormente diseñe y seleccione una estrategia de eliminación de estaciones que produzca el mayor impacto en la red. NOTAS:
Utilice R o Python, según su preferencia.
Consigne el paso a paso realizado en un archivo Jupyter Notebook y/o R Markdown
Elabore una presentación en PowerPoint en la que presente las principales conclusiones del ejercicio. Máximo 5 slides. Se adjuntan los datos de las estaciones y la red de metro de Londres.

https://github.com/safreita1/TIGER

- Plan Carga de la data
- importacion de librerias
- seleccion de métricas a utilizar

## Importación de librerías

In [50]:
import colorsys
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from collections import Counter
from bokeh.plotting import figure, show
from bokeh.resources import CDN
from bokeh.io import output_notebook
from statistics import mean
import statistics

In [122]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)
output_notebook( resources=CDN )

Loading BokehJS ...

## Carga de los datos

In [25]:
lines       = pd.read_csv('london.lines.csv', index_col=0)
stations    = pd.read_csv('london.stations.csv', index_col=0)
connections = pd.read_csv('london.connections.csv')

In [26]:
lines.head()

,name,colour,stripe
line,,,
1,Bakerloo Line,AE6017,NaN
3,Circle Line,FFE02B,NaN
6,Hammersmith & City Line,F491A8,NaN
7,Jubilee Line,949699,NaN
11,Victoria Line,0A9CDA,NaN


In [27]:
stations.head(3)

,latitude,longitude,name,display_name,zone,total_lines,rail
id,,,,,,,
1,51.50280,-0.28010,Acton Town,Acton<br />Town,3.00000,2,0
2,51.51430,-0.07550,Aldgate,NaN,1.00000,2,0
3,51.51540,-0.07260,Aldgate East,Aldgate<br />East,1.00000,2,0


In [28]:
connections.head(3)

,station1,station2,line,time
0,11,163,1,1
1,11,212,1,2
2,49,87,1,1


# Exploracion data


No es un grafo dirigido (DIGRAFO),  posee 302 Estaciones (en adelante nodos) y 406 conexiones (aristas)

In [31]:
print("Número de Estaciones:",len(stations),"Número de Conexciones:", len(connections), "Número de lineas:", len(lines))

Número de Estaciones: 302 Número de Conexciones: 406 Número de lineas: 13


### creación del grafo 

In [94]:
graph = nx.Graph()

for connection_id, connection in connections.iterrows():    
    station1_name = stations.at[connection['station1'],'name']        
    station2_name = stations.at[connection['station2'],'name']
    graph.add_edge(station1_name, station2_name, time = connection['time'])
    
#Se adiciona manualmente el paso entre la estacion Bank y Monument, ya que se transita caminando en la realidad
graph.add_edge('Bank', 'Monument', time = 1)

In [111]:
def pseudocolor(val):
    h = (1.0 - val) * 120 / 360
    r, g, b = colorsys.hsv_to_rgb(h, 1., 1.)
    return r * 255, g * 255, b * 255

In [134]:
normed = stations[['longitude', 'latitude']]
normed = normed - normed.min()
normed = normed / normed.max()
locations = dict(zip(stations['name'], normed[['longitude', 'latitude']].values))
#pageranks = dict(zip(stations['name'], normed['pagerank'].values))

p = figure(
    x_range = (.4,.7),
    y_range = (.2,.5),
    height= 700,
    width= 900,
)
for edge in graph.edges():
    p.line( 
        x= [locations[pt][0] for pt in edge],
        y= [locations[pt][1] for pt in edge],
    )

for node in graph.nodes():
    x = [locations[node][0]]
    y = [locations[node][1]]
    p.circle(
        x, y, 
        radius = .001, 
        fill_color = pseudocolor(1), 
        line_alpha=0)    
show(p)

## Analisis del grafo
Número vertices
Número Aristas
Flujo Total



###  Medidas de centralidad

#### Betweenness_centrality
Numero de veces que la estacion se encuentra dentro del camino más corto

In [95]:
bet_cen=nx.betweenness_centrality(graph)

In [96]:
dfBet_cen=pd.DataFrame(bet_cen.items(), columns=['Estacion', 'betweenness_centrality'])
dfBet_cen=dfBet_cen.sort_values(by=['betweenness_centrality'],ascending=False)
dfBet_cen=dfBet_cen.reset_index()
dfBet_cen

,index,Estacion,betweenness_centrality
0,183,Green Park,0.35291
1,25,Bank,0.33612
2,10,Waterloo,0.33575
3,0,Baker Street,0.31438
4,86,Westminster,0.30133
...,...,...,...
297,128,Richmond,0.00000
298,242,Mornington Crescent,0.00000
299,139,Beckton,0.00000
300,134,Wimbledon,0.00000


#### Closeness centrality
Promedio de las distancias del nodo hacia los demas

In [97]:
clo_cen = nx.closeness_centrality(graph)
dfClo_cen=pd.DataFrame(clo_cen.items(), columns=['Estacion', 'closeness_centrality'])
dfClo_cen=dfClo_cen.sort_values(by=['closeness_centrality'],ascending=False)
dfBet_cen=dfClo_cen.reset_index()
dfBet_cen

,index,Estacion,closeness_centrality
0,183,Green Park,0.11622
1,86,Westminster,0.11295
2,33,Bond Street,0.11269
3,24,Oxford Circus,0.11152
4,10,Waterloo,0.11103
...,...,...,...
297,285,Heathrow Terminal 4,0.04459
298,284,"Heathrow Terminals 1, 2 & 3",0.04459
299,14,Harrow & Wealdston,0.04434
300,127,Upminster Bridge,0.04432


####  Eigenvector  Vector Propio

Influencia del nodo en la red,  nodos conectados a nodos que estan bien conectados

In [98]:
# Eigenvector centrality
eig_cen = nx.eigenvector_centrality(graph, max_iter=1000)

In [99]:
dfEig_cen=pd.DataFrame(eig_cen.items(), columns=['Estacion', 'eigenvector_centrality'])
dfEig_cen=dfEig_cen.sort_values(by=['eigenvector_centrality'],ascending=False)
dfEig_cen=dfEig_cen.reset_index()
dfEig_cen

,index,Estacion,eigenvector_centrality
0,24,Oxford Circus,0.38175
1,183,Green Park,0.38040
2,5,Picadilly Circus,0.30075
3,33,Bond Street,0.27175
4,86,Westminster,0.22929
...,...,...,...
297,284,"Heathrow Terminals 1, 2 & 3",0.00000
298,127,Upminster Bridge,0.00000
299,14,Harrow & Wealdston,0.00000
300,71,West Ruislip,0.00000


In [13]:
centralidad=dfBet_cen.merge(dfClo_cen, on='Estacion', how='left')

##### FUNCION PARA IDENTIFICAR LOS NODOS CENTRALES

###  Evaluación del grado (número de conexiones de las estaciones)

### Weighted Degree
Número de conexiones de los nodos

In [101]:
dfGradosTime=pd.DataFrame(graph.degree(weight='time'), columns=['Estacion', 'Grado'])
dfGradosTime=dfGradosTime.sort_values(by=['Grado'],ascending=False)
dfGradosTime=dfGradosTime.reset_index()
dfGradosTime

,index,Estacion,Grado
0,0,Baker Street,21
1,88,King's Cross St. Pancras,21
2,196,Wembley Park,18
3,199,Chalfont & Latimer,16
4,191,Finchley Road,16
...,...,...,...
297,14,Harrow & Wealdston,2
298,279,Covent Garden,2
299,168,Rotherhithe,2
300,166,Tower Gateway,2


# PageRank

Algoritmo de Google construido para análizar páginas Web. Funciona para validar la relevancia de los nodos en una red

In [68]:
pagerank = nx.pagerank_numpy(graph)
pagerank = pd.DataFrame(pagerank.items(), columns=['name', 'pagerank'])
pagerank = pagerank.sort_values(by='pagerank', ascending=False)
pagerank = pagerank.reset_index()
pagerank

,index,name,pagerank
0,88,King's Cross St. Pancras,0.00791
1,0,Baker Street,0.00761
2,25,Bank,0.00714
3,114,Earl's Court,0.00705
4,7,Paddington,0.00618
...,...,...,...
297,170,New Cross,0.00182
298,297,Brixton,0.00167
299,173,Shoreditch,0.00152
300,115,Kensington (Olympia),0.00150


### HITS

In [70]:
hits = nx.hits_scipy(graph, max_iter=1000)[0]
hits = pd.DataFrame(hits.items(), columns=['name', 'hits'])
hits = hits.sort_values(by='hits', ascending=False)
hits = hits.reset_index()
hits

,index,name,hits
0,24,Oxford Circus,0.05974
1,183,Green Park,0.05952
2,5,Picadilly Circus,0.04706
3,33,Bond Street,0.04253
4,86,Westminster,0.03586
...,...,...,...
297,284,"Heathrow Terminals 1, 2 & 3",0.00000
298,127,Upminster Bridge,0.00000
299,14,Harrow & Wealdston,0.00000
300,71,West Ruislip,0.00000


## Medidas robustes de la red

Agrupando las medidas anteriores se pueden establecer medidas generales de la red que servirán para evaluar las acciones de ataque sobre ella

In [34]:
print("Prom. centralidad de intermediación de la red", dfBet_cen['betweenness_centrality'].mean())
print("Prom. centralidad de cercanía de la red", dfClo_cen['closeness_centrality'].mean())
print("Prom. centralidad de vector propio de la red", dfEig_cen['eigenvector_centrality'].mean())
print("Prom. centralidad de grado de la red", dfGradosTime['Grado'].mean())
print("Prom. centralidad de pagerank de la red", pagerank['pagerank'].mean())

Prom. centralidad de intermediación de la red 0.04355386386804838
Prom. centralidad de cercanía de la red 0.0756830733770901
Prom. centralidad de vector propio de la red 0.02117394181882507
Prom. centralidad de grado de la red 5.377483443708609
Prom. centralidad de pagerank de la red 0.003311258278145694


### Ataques

Para definir los nodos críticos que al ser atacados generaran mayor impacto en la red se procedera a elimiar nodo y a valorar la robustes general de la red.

se probaran 2 estratégias:
1. Reducir la conectividad de los nodos provocando la menor centralidad de grado posible
2. Reducir la intermediacione de la red prolongando los tiempos de desplazamiento


####  Incialmente se replica el grafo 

In [103]:
graphAttack = nx.Graph()
for connection_id, connection in connections.iterrows():    
    station1_name = stations.at[connection['station1'],'name']        
    station2_name = stations.at[connection['station2'],'name']
    graphAttack.add_edge(station1_name, station2_name, time = connection['time'])
    
#Se adiciona manualmente el paso entre la estacion Bank y Monument, ya que se transita caminando en la realidad
graphAttack.add_edge('Bank', 'Monument', time = 1)

In [104]:
#remover nodo con mayor conectividad    
graphAttack.remove_node("King's Cross St. Pancras")
#calcular medida
medida=pd.DataFrame(graphAttack.degree(weight='time'), columns=['Estacion', 'Grado'])['Grado'].mean()
print("Al atacar King's Cross St. Pancras la centralidad de grado es: ",medida)

Al atacar King's Cross St. Pancras la centralidad de grado es:  5.255813953488372


In [ ]:
#remover nodo con mayor conectividad    
graphAttack.remove_node("King's Cross St. Pancras")
#calcular medida
medida=pd.DataFrame(graphAttack.degree(weight='time'), columns=['Estacion', 'Grado'])['Grado'].mean()
print("Al atacar King's Cross St. Pancras la centralidad de grado es: ",medida)

In [105]:
#remover nodo con mayor conectividad    
graphAttack.remove_node("Baker Street")
#calcular medida
medida=pd.DataFrame(graphAttack.degree(weight='time'), columns=['Estacion', 'Grado'])['Grado'].mean()
print("Al atacar Baker Street la centralidad de grado es: ",medida)

Al atacar Baker Street la centralidad de grado es:  5.133333333333334


### Ataque a la centralidad intemedia

In [108]:
graphAttack = nx.Graph()
for connection_id, connection in connections.iterrows():    
    station1_name = stations.at[connection['station1'],'name']        
    station2_name = stations.at[connection['station2'],'name']
    graphAttack.add_edge(station1_name, station2_name, time = connection['time'])
    
#Se adiciona manualmente el paso entre la estacion Bank y Monument, ya que se transita caminando en la realidad
graphAttack.add_edge('Bank', 'Monument', time = 1)

In [109]:
#remover nodo con mayor conectividad    
graphAttack.remove_node("Green Park")
#calcular medida
medida=pd.DataFrame(nx.betweenness_centrality(graphAttack).items(), columns=['Estacion', 'betweenness_centrality'])['betweenness_centrality'].mean()
print("Al atacar Green Park: ",medida)

Al atacar Green Pars:  0.0452796142179357


In [110]:
#remover nodo con mayor conectividad    
graphAttack.remove_node("Bank")
#calcular medida
medida=pd.DataFrame(nx.betweenness_centrality(graphAttack).items(), columns=['Estacion', 'betweenness_centrality'])['betweenness_centrality'].mean()
print("Al atacar Bank: ",medida)

Al atacar Bank:  0.04739130434782609
